In [12]:
import os
import numpy as np
import pandas as pd
import torch
from transformers import Trainer,TrainingArguments,AutoModelForImageClassification,AutoImageProcessor
from datasets import load_dataset
from sklearn.metrics import accuracy_score,precision_score

In [2]:
model_id = ''

In [ ]:
# instantiate the image tookenizer
feature_extractor = AutoImageProcessor.from_pretrained(model_id,trust_remote_code=True)

In [ ]:
# this block is for preprocessing task 

def preprocess(batch):
    img = [x.convert('RGB') for x in batch['image']]
    inps = feature_extractor(img,return_tensors='pt')
    batch['pixel_values'] = inps['pixel_values']
    # batch['label'] = batch['label']
    return batch

In [ ]:
dataset = load_dataset('',split='train')

In [ ]:
dataset = dataset.map(preprocess,batched=True)

In [ ]:
dataset.set_format(type='torch',columns=['pixel_values'])

In [4]:
id2label = {
    0: 'bright dune',
    1: 'crater',
    2: 'dark dune',
    3: 'impact ejecta',
    4: 'other',
    5: 'slope streak',
    6: 'spider',
    7: 'swiss cheese'
}


In [5]:
label2id = {
    'bright dune': 0,
    'crater': 1,
    'dark dune': 2,
    'impact ejecta': 3,
    'other': 4,
    'slope streak': 5,
    'spider': 6,
    'swiss cheese': 7
}


In [ ]:
#instantiate the model 
model = AutoModelForImageClassification.from_pretrained(model_id,num_labels=8,id2label=id2label,label2id=label2id,ignore_mismatched_sizes=True,trust_remote_code=True)

In [8]:
device = torch.device('cuda')
device

device(type='cuda')

In [9]:
model.to(device)

NameError: name 'model' is not defined

In [10]:
training_args = TrainingArguments(
    output_dir='./output_results_vit',
    per_device_eval_batch_size=8,
) 

In [11]:
def compute_metrics(preds):
    labels = preds.label_ids
    preds = np.argmax(preds.predictions,axis=1)
    acc = accuracy_score(labels,preds)

    prec = precision_score(labels,preds,average='weighted')

    return {'accuracy':acc,'precision':prec}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
preds = trainer.predict()